<a href="https://colab.research.google.com/github/youcef-4/VisDrone/blob/main/tph_yolov5_on_VisDrone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/youcef-4/VisDrone.git

Cloning into 'VisDrone'...
remote: Enumerating objects: 258, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 258 (delta 119), reused 244 (delta 112), pack-reused 0
Receiving objects: 100% (258/258), 9.28 MiB | 14.50 MiB/s, done.
Resolving deltas: 100% (119/119), done.


In [3]:
!cd tph-yolov5

/bin/bash: line 0: cd: tph-yolov5: No such file or directory


In [4]:
!pip install -r /content/VisDrone/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 25.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=4df4b75c9a66a166f853ba4fa3b2e2e6f6309606110a9a0085cb148f6e9158ba
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!python /content/VisDrone/VisDrone2YOLO_lable.py

548


In [ ]:
#!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

In [ ]:
#!sudo update-alternatives --config python3

There are 3 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                 Priority   Status
------------------------------------------------------------
  0            /usr/bin/python3.10   2         auto mode
  1            /usr/bin/python3.10   2         manual mode
* 2            /usr/bin/python3.8    1         manual mode
  3            /usr/bin/python3.9    2         manual mode

Press <enter> to keep the current choice[*], or type selection number: ^C


In [ ]:
#! python --version

Python 3.9.1


In [ ]:
#!pip3 install clean-fid numba numpy torch==2.0.0+cu118 torchvision --force-reinstall --extra-index-url https://download.pytorch.org/whl/cu118

In [101]:
#print(torch.__version__)

2.0.0+cu118


In [97]:
#!pip install opencv-python>=4.1.2

In [11]:
!python /content/VisDrone/val.py --weights /content/drive/MyDrive/VisDrone/yolov5l-xs-2.pt --img 1996 --data /content/VisDrone/data/VisDrone.yaml --augment --save-txt  --save-conf --task val --batch-size 8 --verbose --name v5l-xs

val: data=/content/VisDrone/data/VisDrone.yaml, weights=['/content/drive/MyDrive/VisDrone/yolov5l-xs-2.pt'], batch_size=8, imgsz=1996, conf_thres=0.001, iou_thres=0.6, task=val, device=, single_cls=False, augment=True, verbose=True, save_txt=True, save_hybrid=False, save_conf=True, save_json=False, project=VisDrone/runs/val, name=v5l-xs, exist_ok=False, half=False
YOLOv5 🚀 4caad83 torch 2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model Summary: 589 layers, 68030516 parameters, 0 gradients, 151.8 GFLOPs
val: Scanning '../datasets/VisDrone/VisDrone2019-DET-val/labels.cache' images and labels... 548 found, 0 missing, 0 empty, 0 corrupted: 100% 548/548 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 69/69 [16:37<00:00, 14.45s/it]
                 all        548      38759       0.66      0.603      0.632       0.41
          pedestrian        548       8844      0.718       0.71      0.747      0.409
             

In [13]:
!wandb disabled
!python /content/VisDrone/train.py --img 1536 --adam --batch 3 --epochs 80 --data /content/VisDrone/data/VisDrone.yaml --weights yolov5l.pt --hy /content/VisDrone/data/hyps/hyp.VisDrone.yaml --cfg /content/VisDrone/models/yolov5l-xs-tph.yaml --name v5l-xs-tph
#!python /content/VisDrone/train.py --img 1536 --adam --batch 3 --epochs 80 --data /content/VisDrone/data/VisDrone.yaml --weights yolov5l.pt --hy /content/VisDrone/data/hyps/hyp.VisDrone.yaml --cfg /content/VisDrone/models/yolov5l-tph-plus.yaml --name v5l-tph-plus

W&B disabled.
2023-05-16 18:55:15.943394: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 18:55:16.926567: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=yolov5l.pt, cfg=/content/VisDrone/models/yolov5l-xs-tph.yaml, data=/content/VisDrone/data/VisDrone.yaml, hyp=/content/VisDrone/data/hyps/hyp.VisDrone.yaml, epochs=80, batch_size=3, imgsz=1536, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=True, sync_bn=False, workers=8, project=VisDrone/runs/train, name=v5l-xs-tph, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=10